<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_MultiCast_v4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---


![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png) ||
[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

#Create Charts and store in MongoDB

#Setup

##Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [1]:
!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
!pip -qq install python_string_utils
!pip -qq install python-docx                                #https://python-docx.readthedocs.io/en/latest/
#!pip install pyswisseph==2.10.2.0.dev1
!apt install mongodb > /dev/null

     |████████████████████████████████| 521 kB 5.1 MB/s 
     |████████████████████████████████| 5.6 MB 5.0 MB/s 




In [2]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions



In [17]:
# ---------------------------
import p21
import p21utils
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
importlib.reload(p21utils)

p21utils imported


<module 'p21utils' from '/content/p21utils.py'>

In [4]:
#Download Swiss Ephemeris Files 
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe
#------------------------------
# Configure SWE with appropriate parameters
#
p21utils.C01_configSWE()

In [ ]:
#!cat p21utils.py

##Connect to MongoDB

In [5]:
from pymongo import MongoClient
import pandas as pd
import dateutil
import string_utils

In [6]:
from google.colab import drive
drive.mount('/content/drive')
# --------------------------------------------
# copy credentials file
!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB.py credentials.py
# -----------------------------------------------

from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI

# -----------------------------------------------
mongoURI = rMongoURI                                   # for remote server
#mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() 

Mounted at /content/drive
btncgfi3a5bxyea


['btncgfi3a5bxyea']

In [7]:
db = client.btncgfi3a5bxyea
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
pd.DataFrame(list(db.list_collections()))
# empty list '[]' means that there are not collections in database

,name,type,options,info,idIndex
0,khawnaa42,collection,{},"{'readOnly': False, 'uuid': b'\xc5b\xc6\x89\xf...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'n..."
1,restaurants,collection,{},"{'readOnly': False, 'uuid': b""\x8b\x1fUf\x80\x...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'n..."
2,khona21_43,collection,{},"{'readOnly': False, 'uuid': b';\xe0&\x8e\tLGn\...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'n..."


In [18]:
db = client.btncgfi3a5bxyea
#db.p21Charts.drop()
#db.khawnaa42.drop()
db.khona21_43.drop()
db.create_collection('khona21_43')      # https://feminisminindia.com/2019/06/07/khawna-khana-poet-rural-bengal/
                                        # https://en.wikipedia.org/wiki/Khana_(poet)
kollection = db.khona21_43                   # Set the collection to work with

##Read External Data
from CSV files


In [9]:
import pandas as pd
!gdown -O peopleData.csv https://drive.google.com/uc?id=1CVo1gnMZyCD0EZvAoF_wRdTwL7yqIT6L # 5 Person Test Data
#!sed -i '1d' peopleData.csv 
#!cat peopleData.csv
df = pd.read_csv('peopleData.csv')
df.head()

Downloading...
From: https://drive.google.com/uc?id=1CVo1gnMZyCD0EZvAoF_wRdTwL7yqIT6L
To: /content/peopleData.csv
100% 665/665 [00:00<00:00, 1.16MB/s]


,Gender,Voc1_Cat,Voc1_Det,Voc2_Cat,Voc2_Det,Voc3_Cat,Voc3_Det,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_Off,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name
0,M,Engineer,nil,Teacher,nil,PostGraduate,nil,15,10,1961,23:42,h5e30,5.5,22.57,88.37,IST,standard,somu
1,F,Author,nil,Homemaker,nil,PostGraduate,nil,19,12,1965,10:26,h5e30,5.5,22.57,88.37,IST,standard,gopa
2,M,Engineer,nil,Teacher,nil,PostGraduate,nil,9,10,1990,23:09,h5e30,5.5,22.57,88.37,IST,standard,rahool
3,F,Biologist,nil,Scientist,nil,PostGraduate,nil,12,9,1990,0:52,h5e30,5.5,22.87,88.37,IST,standard,nilanjan
4,M,Politician,nil,Administrator,nil,nil,nil,17,9,1950,9:34,h5e30,5.5,23.48,72.36,IST,standard,NDModi


#Insert into MongoDB database
Driver Code

In [19]:
p21.ChartType = 'Rashi'

for person in range(len(df)):
    personData =df.iloc[person]
    p21utils.C02_parsePersonData(personData)            # split the data, extract required data into fields
                                                        # create skeleton chart json doc as p21.chart
    p21utils.C03_convertDates()                         # convert date, time of birth into universal time, calculate Ayanamsha
    p21utils.C04_calculateGrahaPositions()              # calculate the Lon of each Graha along with Retrograde staus

                #Horoscope is defined in terms of Natal Longitudes and Retrogrades
                # for example
                #GLon = {"La":98.5,"Su":178.9,"Mo":250.6,"Ma":196.2,"Me":193.2,"Ju":274.8,"Ve":153.8,"Sa":270.2,"Ra":122.1,"Ke":302.1}
                #GRet = {"La":False,"Su":False,"Mo":False,"Ma":False,"Me":True,"Ju":False,"Ve":False,"Sa":False,"Ra":True,"Ke":True}

    p21utils.C05_buildGLonGRet()                        # create two json docs ( python dicts) to store Graha Lon and Ret status
                                                        # these are stored as global variables p21.GLon, p21.GRet
    p21utils.appendDict(p21.chart,p21.GLonRet)             # add Lon, Ret information to chart
    print('chart >',p21.chart)
    # ----------------------------------------
    p21utils.R11_LocateGrahaInRashi()
    p21utils.C10_DetermineBhavs()                       # determine Houses for person
    p21utils.appendDict(p21.chart,p21.BhavNBhavA)       # add BhavN, BhavA information to chart   
    print('chart >>',p21.chart)    
    # ----------------------------------------    
    p21utils.C11_DetermineLord()                       # determine Lords of Bhavs
    p21utils.appendDict(p21.chart,p21.LordInfo)       #  add information on Lords   
    print('chart >>>',p21.chart)    
    insert_result = kollection.insert_one(p21.chart)    # insert one json doc, chart to database
    #insert_result.acknowledged    # Confirms that insert is successful
    
    print('------------')

chart > {'pid': {'tags': ['Engineer', 'Teacher', 'PostGraduate'], 'ck': 'M1961101523:425.522.5788.37', 'name': 'umso'}, 'GLon': {'La': 98.204, 'Su': 178.826, 'Mo': 250.52, 'Ma': 196.183, 'Me': 193.17, 'Ju': 274.781, 'Ve': 153.753, 'Sa': 270.179, 'Ra': 122.089, 'Ke': 302.089}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': True, 'Ju': False, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}}
chart >> {'pid': {'tags': ['Engineer', 'Teacher', 'PostGraduate'], 'ck': 'M1961101523:425.522.5788.37', 'name': 'umso'}, 'GLon': {'La': 98.204, 'Su': 178.826, 'Mo': 250.52, 'Ma': 196.183, 'Me': 193.17, 'Ju': 274.781, 'Ve': 153.753, 'Sa': 270.179, 'Ra': 122.089, 'Ke': 302.089}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': True, 'Ju': False, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}, 'BhavN': [' ', 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3], 'BhavA': [' ', 'Can', 'Leo', 'Vir', 'Lib', 'Sco', 'Sag', 'Cap', 'Acq', 'Pis', 'Ari', 'Tau', 'Gem']}
Lord :  [' '

In [21]:
# Check data loaded into MongoDb
#c = kollection.find()
#for crnt in c:
#    print(crnt)
pd.DataFrame(list(kollection.find({},{'_id':0})))

,pid,GLon,GRet,BhavN,BhavA,Lord,LordRashiN,LordRashiA,GrahaLordBhav
0,"{'tags': ['Engineer', 'Teacher', 'PostGraduate...","{'La': 98.204, 'Su': 178.826, 'Mo': 250.52, 'M...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]","[ , Can, Leo, Vir, Lib, Sco, Sag, Cap, Acq, Pi...","[ , Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma, Ve...","[ , 9, 6, 7, 6, 7, 10, 10, 10, 10, 7, 6, 7]","[ , Sag, Vir, Lib, Vir, Lib, Cap, Cap, Cap, Ca...","{'Su': [2], 'Mo': [1], 'Ma': [5, 10], 'Me': [3..."
1,"{'tags': ['Author', 'Homemaker', 'PostGraduate...","{'La': 310.415, 'Su': 243.701, 'Mo': 201.001, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[ , Acq, Pis, Ari, Tau, Gem, Can, Leo, Vir, Li...","[ , Sa, Ju, Ma, Ve, Me, Mo, Su, Me, Ve, Ma, Ju...","[ , 11, 3, 10, 10, 8, 7, 9, 8, 10, 10, 3, 11]","[ , Acq, Gem, Cap, Cap, Sco, Lib, Sag, Sco, Ca...","{'Su': [7], 'Mo': [6], 'Ma': [3, 10], 'Me': [5..."
2,"{'tags': ['Engineer', 'Teacher', 'PostGraduate...","{'La': 85.351, 'Su': 172.44, 'Mo': 63.984, 'Ma...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]","[ , Gem, Can, Leo, Vir, Lib, Sco, Sag, Cap, Ac...","[ , Me, Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma...","[ , 6, 3, 6, 6, 6, 2, 4, 9, 9, 4, 2, 6]","[ , Vir, Gem, Vir, Vir, Vir, Tau, Can, Sag, Sa...","{'Su': [3], 'Mo': [2], 'Ma': [6, 11], 'Me': [1..."
3,"{'tags': ['Biologist', 'Scientist', 'PostGradu...","{'La': 84.138, 'Su': 145.068, 'Mo': 54.233, 'M...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]","[ , Gem, Can, Leo, Vir, Lib, Sco, Sag, Cap, Ac...","[ , Me, Mo, Su, Me, Ve, Ma, Ju, Sa, Sa, Ju, Ma...","[ , 5, 2, 5, 5, 5, 2, 4, 9, 9, 4, 2, 5]","[ , Leo, Tau, Leo, Leo, Leo, Tau, Can, Sag, Sa...","{'Su': [3], 'Mo': [2], 'Ma': [6, 11], 'Me': [1..."
4,"{'tags': ['Politician', 'Administrator', 'nil'...","{'La': 191.897, 'Su': 150.533, 'Mo': 217.939, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ...","[ , 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6]","[ , Lib, Sco, Sag, Cap, Acq, Pis, Ari, Tau, Ge...","[ , Ve, Ma, Ju, Sa, Sa, Ju, Ma, Ve, Me, Mo, Su...","[ , 5, 8, 11, 5, 5, 11, 8, 5, 6, 8, 6, 6]","[ , Leo, Sco, Acq, Leo, Leo, Acq, Sco, Leo, Vi...","{'Su': [11], 'Mo': [10], 'Ma': [2, 7], 'Me': [..."


In [ ]:
#Retrieves Charts based on tags
pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})))
#c = kollection.find({'pid.tags': {"$in": ['Scientist','Author']}},{'pid.tags':1,'GLon':1,'GRet':1,'_id':0})
#for crnt in c:
#    print(crnt)

,pid,GLon,GRet
0,"{'tags': ['Author', 'Homemaker', 'PostGraduate']}","{'La': 310.415, 'Su': 243.701, 'Mo': 201.001, ...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ..."
1,"{'tags': ['Biologist', 'Scientist', 'PostGradu...","{'La': 84.138, 'Su': 145.068, 'Mo': 54.233, 'M...","{'La': False, 'Su': False, 'Mo': False, 'Ma': ..."


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)